In [ ]:
# Install libreries

#!pip install python-dotenv openai anthropic gradio

In [ ]:
# Imports

from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import os

In [ ]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [ ]:
openai = OpenAI()
OPENAI_MODEL = "gpt-4o-mini"

In [ ]:
python_system_prompt = """
You are an intelligent assistant specializing in software documentation and code understanding. 
Your task is to analyze given Python code snippets, understand their functionality, and produce detailed and accurate docstrings and inline comments. 
Follow these guidelines: 
1. Use triple double quotes (\"\"\") for Python docstrings. 
2. Follow the Google-style docstring format, including: 
   - A concise description of the function or class. 
   - Args: A list describing all parameters with types and purposes. 
   - Returns: A description of the return value, including the type. 
   - Example: An example of the function and the output. 
3. Add inline comments for non-obvious logic or important steps in the code. 
   - Inline comments should be concise and informative. 
4. Ensure all docstrings and comments adhere to Python's PEP 8 guidelines. 
5. Use clear and professional language, avoiding over-explaining obvious code. 
For example, if the input is: 
def add_numbers(a, b): 
    return a + b 
The output should be: 
def add_numbers(a, b): 
    \"\"\"Adds two numbers and returns the result. 
    Args: 
        a (int): The first number. 
        b (int): The second number. 
    Returns: 
        int: The sum of the two numbers. 
    Example: 
        >>> add_numbers(1, 2) 
        3 
    \"\"\" 
    return a + b  # Add the two numbers and return the result
"""


In [ ]:
java_system_prompt = """
You are an intelligent assistant specializing in software documentation and code understanding.
Your task is to analyze given Java code snippets, understand their functionality, and produce detailed and accurate Javadoc-style comments.
Follow these guidelines:

1. Use standard Javadoc syntax:
   - Start each comment block with /** and end with */.
   - Use @param for parameter descriptions.
   - Use @return to describe return values.
   - Use @throws to describe exceptions thrown by the method, if any.

2. Ensure comments are concise but provide enough detail for a developer to understand the purpose and functionality of the code.

Example Input:
public int addNumbers(int a, int b) {
    return a + b;
}

Example Output:
/**
 * Adds two numbers and returns the result.
 *
 * @param a the first number
 * @param b the second number
 * @return the sum of the two numbers
 */
public int addNumbers(int a, int b) {
    return a + b;
}
"""


In [ ]:
def user_prompt_for_python(python_code):
    user_prompt = "Analyze the following Python code snippet and add detailed docstrings and inline comments to improve readability and maintainability. "
    user_prompt += "Ensure the docstrings follow the Google-style format with sections such as Description, Args, Returns, and Raises if applicable. "
    user_prompt += "Respond only with the updated Python code, including the added docstrings and inline comments, and nothing else.\n\n"
    user_prompt += python_code
    return user_prompt


In [ ]:
def messages_for_python(python):
    return [
        {"role": "system", "content": python_system_prompt},
        {"role": "user", "content": user_prompt_for_python(python)}
    ]

In [ ]:
def user_prompt_for_java(java_code):
    user_prompt = "Analyze the following Java code snippet and add detailed Javadoc-style comments to improve readability and maintainability. "
    user_prompt += "Ensure the comments use standard Javadoc syntax with @param, @return, and @throws sections where applicable. "
    user_prompt += "Respond only with the updated Java code, including the added comments, and nothing else.\n\n"
    user_prompt += java_code
    return user_prompt


In [ ]:
def messages_for_java(java_code):
    return [
        {"role": "system", "content": java_system_prompt},
        {"role": "user", "content": user_prompt_for_java(java_code)}
    ]


In [ ]:
def documentation_python(python):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for_python(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```python\n','').replace('```','')

In [ ]:
def documentation_java(java_code):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for_java(java_code), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```java\n', '').replace('```', '')


In [ ]:
def documentation(code, language):
    if language == "Python":
        result = documentation_python(code)
    elif language == "Java":
        result = documentation_java(code)
    else:
        return "Language not supported yet."
    for stream_so_far in result:
        yield stream_so_far


In [ ]:
simple_function_python = """
def calculate_area(length, width):
    return length * width
"""

In [ ]:
simple_function_java = """
public int multiply(int a, int b) {
    return a * b;
}
"""

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        language = gr.Dropdown(["Python", "Java"], label="Select language", value="Python")
    with gr.Row():
        code_input = gr.Code(label="Code snippets:", lines=10, value=simple_function_python, language="python")
        code_doc = gr.Code(label="Documented code:", language="python")
    with gr.Row():
        convert = gr.Button("Document code")
    
    convert.click(documentation, inputs=[code_input, language], outputs=[code_doc])
    
ui.launch(inbrowser=True, share=True)
